In [1]:
#%matplotlib inline
import numpy as np
import random as rm
import math
import matplotlib.pyplot as plt
import statistics as stat

### copied from aludert's data faker

In [2]:
def data_faker_2(N, m, b, c=1, gamma=0.0000001, rand_div = 'yes'):
    
    if rand_div == 'yes':
        
        cosine_sign_choice = rm.random()
        
        if cosine_sign_choice < 0.5:
            cosine_sign = -1
        else:
            cosine_sign = 1
            
        cosine = cosine_sign * rm.random()
        
        angle = math.acos(cosine)
        
        m = math.sin(angle)/math.cos(angle)
        
        intercep_sign_choice = rm.random()
        
        if intercep_sign_choice < 0.5:
            b_sign = -1
        else:
            b_sign = 1
            
        b = b_sign * rm.random()
    
    alpha = math.atan(m)
    bd = gamma / math.cos(alpha)
    
    if 0<=abs(alpha)<45:
        gamma_max = (1/math.cos(alpha))*c
    elif 45<=abs(alpha)<90:
        gamma_max = (1/math.sin(alpha))*c
        
    points = np.empty([N,3])
        
    if gamma > gamma_max:
        print('gamma is greater than allowed gamma_max for this slope')
        return points, m, b
    
    for i in range(N):
        
        good = 0
        
        while good == 0:
        
            x_sign_choice = rm.random()

            if x_sign_choice < 0.5:
                x_sign = -1
            else:
                x_sign = 1

            x_i = x_sign * rm.random() *c
            #---------------------------------------

            y_sign_choice = rm.random()

            if y_sign_choice < 0.5:
                y_sign = -1
            else:
                y_sign = 1

            y_i = y_sign * rm.random() *c
            #----------------------------------------
            
            if y_i < m*x_i+b:
                l_i = 1
            elif y_i > m*x_i+b:
                l_i = -1
            
            if l_i == 1 and y_i < m*x_i+(b-bd):
                good = 1
            elif l_i ==-1 and y_i > m*x_i+(b+bd):
                good = 1
                
        points[i,0] = x_i
        points[i,1] = y_i
        points[i,2] = l_i
         
    return points, m, b

In [3]:
step_function = lambda f: -1 if f <= 0 else 1

### Perceptron algorithm

In [4]:
def perceptron(points, w=(0,0,0), eta=0.5, n_iter=10000, video='no', m=0, b=0, vid_dir = './vid'):
    
    test_training = np.ones([len(points),4])
    test_training[:,1:] =  points
    
    misclassed = True
    num_slope_adjusts = 0
    n = 0
    num_points_tested = 0
    slope = 0
    intercept = 0
    
    while (misclassed and n < n_iter):
        n += 1
        wrong_line = 0
        
        for test_val in test_training:
            point = test_val[:3]
            desired = test_val[3]
            result = np.dot(w, point)
            f = step_function(result)
            num_points_tested += 1
            
            if f != desired:
                wrong_line += 1
                num_slope_adjusts += 1
                w += eta * desired * point
                
                if w[2] != 0:
                    slope = -(w[1] / w[2])
                    intercept = -(w[0] / w[2])
                else:
                    slope = 0
                    intercept = 0
                
                if video == 'yes':
                        scatter_plot_vid(points, m, b, 
                                         slope, intercept, 
                                         num_points_tested, vid_dir = './vid')
        
        if wrong_line == 0: 
            misclassed = False

        
    return slope, intercept, n, num_slope_adjusts

### Exp 1: Number of weight vector adjustments vs Number of data points, N (eta=0.5, c=10)

In [7]:
N = 0
N_list = []
k_ave_list = []
k_SDM_list = []
i = 0
num_trials = 1000
for N in range(10, 10000, 1000):
    k_list = []
    N_list.append(N)
    for i in range(num_trials):
        points, m, b = data_faker_2(N, 1, 0, c=10, rand_div='no')
        p_m, p_b, n, k = perceptron(points)
        k_list.append(k)
    k_ave = stat.mean(k_list)
    k_SDM = stat.stdev(k_list) / math.sqrt(num_trials)
    k_ave_list.append(k_ave)
    k_SDM_list.append(k_SDM)
    
np.save('N_list.npy',N_list)
np.save('k_ave_list.npy',k_ave_list)
np.save('k_SDM_list.npy',k_SDM_list)

In [ ]:
# np.load()

In [8]:
time_plot_1 = plt.errorbar(N_list, k_ave_list, yerr=k_SDM_list, fmt='o', color='r')
plt.title('')
plt.xlabel('Number of data points')
plt.ylabel('Number of weight vector adjustments')
plt.savefig('Exp_1_large_N.png')